# BentoML Example:  Deploy to Kubernetes Cluster


# !IMPORTANT - this notebook is still working-in-progress, not everything works

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a serverless REST API endpoint to any Kubernetes cluster. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=&t=event&ec=nb&ea=open&dp=deploy-with-kubernetes&dt=deploy-with-kubernetes)

In [ ]:
!pip install -I bentoml
!pip install tensorflow==1.13.1
!pip install numpy==1.16.1

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import bentoml

## Download the IMDB dataset

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

## Build the model

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

## Create a validation set

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Train the model

In [ ]:
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## Evaluate the model

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

In [ ]:
model.summary()

# Define ML service with BentoML

In [ ]:
%%writefile text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import TfKerasModelArtifact, PickleArtifact
from bentoml.handlers import JsonHandler

@artifacts([
    TfKerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(requirements_txt="""tensorflow>=1.13.1 
numpy>=1.16.2
pandas>=0.24.2
""")
class TextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        """
        """
        text = parsed_json['text']
        
        sequence = keras.preprocessing.text.hashing_trick(
            text,
            256,
            hash_function=self.word_to_index,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True,
            split=' ')

        return self.artifacts.model.predict(np.expand_dims(sequence, 0))

# Save BentoML service archive

In [ ]:
from text_classification_service import TextClassificationService

svc = TextClassificationService.pack(model=model, word_index=word_index)
saved_path = svc.save('/tmp/bento')
print(saved_path)

### Test packed BentoML service

In [ ]:
svc.predict({ 'text': 'the best movie I have ever seen' })

In [ ]:
result = svc.predict({ 'text': 'This is a bad movie' })
print(result.tolist())

In [ ]:
import os
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

In [ ]:
# Working in progress